# Initial experiments Part 2

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from dataset import QuickDrawDataset
from utils import AbsolutePenPositionTokenizer
from prepare_data import stroke_to_rdp
from tqdm import tqdm
import pickle

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

seed = 42
torch.manual_seed(seed)
if device == "cuda":
    torch.cuda.manual_seed_all(seed)

Using device: cuda


In [ ]:
labels = ["cat"]

training_data = QuickDrawDataset(
    labels=labels,
)

tokenizer = AbsolutePenPositionTokenizer(bins=64)


class SketchDataset(Dataset):
    def __init__(
        self,
        svg_list,
        tokenizer,
        max_len=200,
        cache_file="sketch_rdp_tokenized_200_dataset.pkl",
    ):
        self.data = []
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad_id = tokenizer.vocab["PAD"]

        # Try to load from cache
        try:
            with open(cache_file, "rb") as f:
                self.data = pickle.load(f)
            print(f"Loaded tokenized data from {cache_file}")
        except FileNotFoundError:
            try:
                for svg in tqdm(svg_list, desc="Tokenizing SVGs"):
                    # use RDP to reduce number of points in SVG
                    svg = stroke_to_rdp(svg, epsilon=2.0)  # tuning
                    tokens = tokenizer.encode(svg)
                    # Truncate + Pad
                    tokens = tokens[:max_len]
                    tokens = tokens + [self.pad_id] * (max_len - len(tokens))
                    self.data.append(tokens)

            except Exception as e:
                print(f"Error processing SVG: {e}")
                pass

            # Save to cache
            with open(cache_file, "wb") as f:
                pickle.dump(self.data, f)
            print(f"Saved tokenized data to {cache_file}")

    def __getitem__(self, idx):
        seq = self.data[idx]
        input_ids = torch.tensor(seq[:-1])
        target_ids = torch.tensor(seq[1:])
        return input_ids, target_ids

    def __len__(self):
        return len(self.data)


dataset = SketchDataset(training_data, tokenizer)

Tokenizing SVGs: 100%|██████████| 103031/103031 [02:01<00:00, 845.89it/s]


Saved tokenized data to sketch_rdp_tokenized_dataset.pkl


In [ ]:
def generate_square_subsequent_mask(sz: int):
    """Causal mask to stop attention to future positions"""
    return torch.triu(torch.ones(sz, sz), diagonal=1).bool()


class SketchTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=8, num_layers=6, max_len=200):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_len = max_len

        self.embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Embedding(max_len, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=4 * d_model
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        """
        x: (batch, seq_len) input tokens
        Returns: (batch, seq_len, vocab_size) logits
        """
        batch_size, seq_len = x.shape
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)

        x = self.embed(x) + self.pos_embed(positions)  # (batch, seq_len, d_model)
        x = x.transpose(0, 1)  # -> (seq_len, batch, d_model)

        # causal mask (seq_len, seq_len)
        mask = generate_square_subsequent_mask(seq_len).to(x.device)

        x = self.transformer(x, mask=mask)  # (seq_len, batch, d_model)
        x = x.transpose(0, 1)  # back to (batch, seq_len, d_model)

        logits = self.fc_out(x)  # (batch, seq_len, vocab_size)
        return logits


def train_model(model, dataloader, vocab_size, epochs=10, lr=1e-4, device="cuda"):
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # ignore pad token

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for input_ids, target_ids in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            input_ids, target_ids = input_ids.to(device), target_ids.to(device)

            # Transformer expects shape (seq_len, batch, d_model)
            logits = model(input_ids)  # (seq_len, batch, vocab_size)
            loss = criterion(logits.view(-1, vocab_size), target_ids.view(-1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss/len(dataloader):.4f}")


dataloader = DataLoader(dataset, batch_size=128, shuffle=True, pin_memory=True)
model = SketchTransformer(
    vocab_size=len(tokenizer.vocab), d_model=256, nhead=8, num_layers=6
)
train_model(
    model,
    dataloader,
    vocab_size=len(tokenizer.vocab),
    epochs=40,
    lr=1e-4,
    device=device,
)

c:\Users\matth\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
Epoch 1/40: 100%|██████████| 805/805 [01:10<00:00, 11.41it/s]


Epoch 1 Loss: 3.2771


Epoch 2/40: 100%|██████████| 805/805 [01:11<00:00, 11.27it/s]


Epoch 2 Loss: 2.8960


Epoch 3/40: 100%|██████████| 805/805 [01:11<00:00, 11.27it/s]


...

Epoch 9 Loss: 2.1039


Epoch 10/40: 100%|██████████| 805/805 [01:10<00:00, 11.38it/s]


Epoch 10 Loss: 2.0823


Epoch 11/40:   7%|▋         | 54/805 [00:04<01:07, 11.11it/s]


KeyboardInterrupt: 

In [5]:
torch.save(model, "sketch_transformer_model_cat.pth")

In [ ]:
# model = torch.load("sketch_transformer_cat_decoder_checkpoint.pth", map_location=device, weights_only=False)


def sample_sequence(
    model,
    start_token,
    max_len=200,
    temperature=1.0,
    greedy=False,
    eos_id=None,
    device="cuda",
):
    model.eval()

    tokens = [start_token]
    tokens_tensor = torch.tensor([tokens], device=device)  # (1, 1)

    for _ in range(max_len - 1):
        with torch.no_grad():
            logits = model(tokens_tensor)  # (batch=1, seq_len, vocab_size)
            next_logits = logits[:, -1, :] / temperature  # take last step
            probs = F.softmax(next_logits, dim=-1)

            if greedy:
                next_token = torch.argmax(probs, dim=-1).item()
            else:
                next_token = torch.multinomial(probs, num_samples=1).item()

        tokens.append(next_token)

        # stop if EOS reached
        if eos_id is not None and next_token == eos_id:
            break

        next_token_tensor = torch.tensor([[next_token]], device=device)
        tokens_tensor = torch.cat([tokens_tensor, next_token_tensor], dim=1)

    return tokens


# TODO Top-k filtering, Top-p filtering (nucleus)

start_token = tokenizer.vocab["START"]
eos_token = tokenizer.vocab.get("END", None)

generated = sample_sequence(
    model,
    start_token,
    max_len=200,
    temperature=0.5,
    greedy=False,
    eos_id=eos_token,
    device=device,
)

print("Generated token sequence:", generated)
decoded_sketch = tokenizer.decode(generated, stroke_width=0.4)
print("Decoded sketch:", decoded_sketch)

from IPython.display import HTML, display

display(
    HTML(
        f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated</b><br>{decoded_sketch}</div>'
    )
)

Generated token sequence: [4098, 4096, 1045, 919, 732, 544, 423, 493, 818, 948, 2106, 2679, 2997, 3310, 3371, 3361, 3164, 2968, 2583, 1875, 1365, 4096, 1044, 712, 640, 961, 1413, 1612, 4096, 2385, 2766, 3271, 3650, 3852, 3734, 4096, 1959, 1960, 2087, 4096, 1959, 1574, 743, 4096, 1895, 1324, 688, 4096, 1960, 1005, 4096, 2154, 2984, 4096, 2094, 2161, 2357, 4096, 1501, 1245, 4096, 2204, 2207, 4099]
Decoded sketch: <svg viewBox="0 0 64 64"><g stroke-width="0.4">
<path d="M 16 21 L 14 23 L 11 28 L 8 32 L 6 39 L 7 45 L 12 50 L 14 52 L 32 58 L 41 55 L 46 53 L 51 46 L 52 43 L 52 33 L 49 28 L 46 24 L 40 23 L 29 19 L 21 21" stroke="black" fill="none"/>
<path d="M 16 20 L 11 8 L 10 0 L 15 1 L 22 5 L 25 12" stroke="black" fill="none"/>
<path d="M 37 17 L 43 14 L 51 7 L 57 2 L 60 12 L 58 22" stroke="black" fill="none"/>
<path d="M 30 39 L 30 40 L 32 39" stroke="black" fill="none"/>
<path d="M 30 39 L 24 38 L 11 39" stroke="black" fill="none"/>
<path d="M 29 39 L 20 44 L 10 48" stroke="black" fill="

In [ ]:
def top_p_filtering(logits, p=0.9):
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

    sorted_indices_to_remove = cumulative_probs > p
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = False

    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[:, indices_to_remove] = -float("Inf")
    return logits


def top_k_filtering(logits, k):
    if k <= 0:
        return logits
    top_k = min(k, logits.size(-1))
    values, _ = torch.topk(logits, top_k)
    min_values = values[:, -1].unsqueeze(-1)
    logits[logits < min_values] = -float("Inf")
    return logits


def sample_sequence_feat(
    model,
    start_token,
    max_len=200,
    temperature=1.0,
    top_k=60,
    top_p=0.9,
    greedy=False,
    eos_id=None,
    device="cuda",
):
    model.eval()
    tokens = [start_token]
    tokens_tensor = torch.tensor([tokens], device=device)

    for _ in range(max_len - 1):
        with torch.no_grad():
            logits = model(tokens_tensor)
            next_logits = logits[:, -1, :] / temperature

            # top-k / top-p filtering
            next_logits = top_k_filtering(next_logits, top_k)
            next_logits = top_p_filtering(next_logits, top_p)

            probs = F.softmax(next_logits, dim=-1)

            if greedy:
                next_token = torch.argmax(probs, dim=-1).item()
            else:
                next_token = torch.multinomial(probs, num_samples=1).item()

        tokens.append(next_token)

        if eos_id is not None and next_token == eos_id:
            break

        next_token_tensor = torch.tensor([[next_token]], device=device)
        tokens_tensor = torch.cat([tokens_tensor, next_token_tensor], dim=1)

    return tokens


start_token = tokenizer.vocab["START"]
eos_token = tokenizer.vocab.get("END", None)

generated = sample_sequence_feat(
    model,
    start_token,
    max_len=200,
    temperature=1.0,
    greedy=False,
    eos_id=eos_token,
    device=device,
)

print("Generated token sequence:", generated)
decoded_sketch = tokenizer.decode(generated, stroke_width=0.4)
print("Decoded sketch:", decoded_sketch)

from IPython.display import HTML, display

display(
    HTML(
        f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated</b><br>{decoded_sketch}</div>'
    )
)

Generated token sequence: [4098, 4096, 658, 597, 602, 673, 933, 1257, 1772, 2283, 2475, 2918, 3039, 3035, 3027, 2832, 2254, 1932, 1420, 783, 592, 403, 4096, 786, 657, 523, 512, 836, 1421, 4096, 2315, 2950, 3392, 3713, 3653, 3215, 4096, 1755, 1883, 4096, 1885, 2016, 2016, 1825, 1567, 4096, 2016, 1891, 4096, 1887, 2016, 2144, 4096, 2142, 2268, 2779, 4060, 4096, 2082, 2467, 3044, 4096, 2014, 1629, 1054, 31, 4096, 1759, 1568, 994, 739, 232, 4099]
Decoded sketch: <svg viewBox="0 0 64 64"><g stroke-width="0.4">
<path d="M 10 18 L 9 21 L 9 26 L 10 33 L 14 37 L 19 41 L 27 44 L 35 43 L 38 43 L 45 38 L 47 31 L 47 27 L 47 19 L 44 16 L 35 14 L 30 12 L 22 12 L 12 15 L 9 16 L 6 19" stroke="black" fill="none"/>
<path d="M 12 18 L 10 17 L 8 11 L 8 0 L 13 4 L 22 13" stroke="black" fill="none"/>
<path d="M 36 11 L 46 6 L 53 0 L 58 1 L 57 5 L 50 15" stroke="black" fill="none"/>
<path d="M 27 27 L 29 27" stroke="black" fill="none"/>
<path d="M 29 29 L 31 32 L 31 32 L 28 33 L 24 31" stroke="black" fill="no